In [1]:
import pandas as pd
from  more_itertools import unique_everseen
import codecs
import re
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm as tq
from tqdm import tqdm_notebook
tqdm_notebook().pandas()

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def list_to_file(filepath, list_, codec):
    with codecs.open(filepath, 'w', codec) as file:
        for line in list_:
            line = line + '\n'
            file.write(line)
            
def read_list(filepath, codec):
    with codecs.open(filepath, 'r', codec) as file:
        data = file.readlines()
        data = [line.rstrip() for line in data]
    return data

def recursive_flatten_generator(array):
    lst = []
    for i in array:
        if isinstance(i, list):
            lst.extend(recursive_flatten_generator(i))
        else:
            lst.append(i)
    return lst

def normalize_corp(corp_list):
    norm_corp = []
    for item in corp_list:
        words = re.split('\W+', item)
        norm_word = [morph.parse(word)[0].normal_form for word in words]
        norm_corp.append(norm_word)
    norm_corp = [" ".join(item) for item in norm_corp]
    return norm_corp

In [3]:
data = pd.read_csv('../_data/patients_with operations_dataset_clean.txt', encoding='cp1251', sep='\t', engine='python', header=None, error_bad_lines=False,parse_dates=[3])
data.columns = ['index', 'patient', 'episode','date', 'time','clinitian', 'depart', 'hosp','specialty','appoint','document','text','vyp']
oper_to_mod = read_list('../_data/oper_to_mod.txt','utf-8') #Операции от Дмитрия
intra_docs = read_list('../_data/intra_docs.txt','utf-8')
data['text'] = data.apply(lambda x: re.sub(r'293pan\d+', '', x['text']), axis=1)
data['text'] = data.apply(lambda x: re.sub(r'10sup\d+', '', x['text']), axis=1)
data['text'] = data.apply(lambda x: re.sub(r'\s+', ' ', x['text']), axis=1)
data.head()

,index,patient,episode,date,time,clinitian,depart,hosp,specialty,appoint,document,text,vyp
0,0,GACAAGc,GACAAGcAAAAIe10:30-uAA,2009-03-18,10:30b 11:00,Наумова_Людмила_Васильевна,Кар,Центр_Алмазова,Врач-кардиолог,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ДИАГНОЗ_АМБУЛАТОРНЫЙ,ДИАГНОЗ_ПРЕДВАРИТЕЛЬНЫЙ_ОСНОВНОЙ I20.8 Другие ...,выполнено
1,1,GACAAGc,GACAAGcAAAAIxN20288uAA,2009-04-06,10:55,Наумова_Людмила_Васильевна,Кар,Центр_Алмазова,Врач-кардиолог,"Прием_(осмотр,_консультация)_врача-кардиолога_...",ДИАГНОЗ_АМБУЛАТОРНЫЙ,ДИАГНОЗ_ПРЕДВАРИТЕЛЬНЫЙ_ОСНОВНОЙ I20.8 Другие ...,выполнено
2,2,GACAAGc,GACAAGcAAAAIxN20288{AD,2009-04-06,10:55,Наумова_Людмила_Васильевна,Кар,Центр_Алмазова,Врач-кардиолог,"Прием_(осмотр,_консультация)_врача-кардиолога_...",РЕКОМЕНДАЦИИ,"Липидный спектр, калий, креатинин, МНО, сахар,...",выполнено
3,3,GACAAGc,GACAAGcAAAAJN15:00-iAD,2009-04-22,15:00-15:30,Дзюбенко_Наталья_Александровна,ФД,Центр_Алмазова,Врач_ультразвуковой_диагностики,Эхокардиография,ЭХОКАРДИОГРАФИЧЕСКОЕ_ИССЛЕДОВАНИЕ_РЕЗУЛЬТАТ,"ЧСС 64 уд.в мин., Ритм синусовый",выполнено
4,4,GACAAGc,GACAAGcAAAAJN15:00-iAG,2009-04-22,15:00-15:30,Дзюбенко_Наталья_Александровна,ФД,Центр_Алмазова,Врач_ультразвуковой_диагностики,Эхокардиография,ЭХОКАРДИОГРАФИЧЕСКОЕ_ИССЛЕДОВАНИЕ_РЕЗУЛЬТАТ,"Аорта 41 мм стенки уплотнены ,диаметр расшире...",выполнено


In [4]:
intra_docs

['ПРОТОКОЛ_ОПЕРАЦИИ.',
 'ПРОТОКОЛ_ОПЕРАЦИИ',
 'ПРОТОКОЛ_ОПЕРАЦИИ_НА_СЕРДЦЕ',
 'ПРОТОКОЛ_ОПЕРАЦИИ_(ИМПЛАНТАЦИЯ_ИКД) первичная без смены ИКД',
 'ПРОТОКОЛ_ОПЕРАЦИИ_(ИМПЛАНТАЦИЯ_ЭКС) первичная без смены ЭКС',
 'ПРОТОКОЛ_ОПЕРАЦИИ_(НА_СРЕДОСТЕНИИ)',
 'ОПЕРАЦИЯ_НА_СОСУДАХ_ШЕИ._КАРОТИДНАЯ_ЭНДАТЕРЭКТОМИЯ',
 'ОПЕРАЦИЯ_АРИТМОЛОГИЧЕСКАЯ_(ИМПЛАНТАЦИЯ_УСТРОЙСТВ)',
 'ОПЕРАЦИЯ_АРИТМОЛОГИЧЕСКАЯ_(РЧ/РЧА)',
 'ПРОТОКОЛ_ОПЕРАЦИИ_ЗАМЕСТИТЕЛЬНОЙ_ПОЧЕЧНОЙ_ТЕРАПИИ',
 'ПРОТОКОЛ_ОПЕРАЦИИ_(РЧА)',
 'ПРОТОКОЛ_ОПЕРАЦИИ_(РЕВИЗИЯ)',
 'ПРОТОКОЛ_ОПЕРАЦИИ_ЗАМЕСТИТЕЛЬНОЙ_ПОЧЕЧНОЙ_ТЕРАПИИ_(MPS+PE)',
 'ПРОТОКОЛ_ИНТРАОПЕРАЦИОННОГО_НЕЙРОФИЗИОЛОГИЧЕСКОГО_МОНИТОРИНГА',
 'ПРОТОКОЛ_ОПЕРАЦИИ_ЭНДОПРОТЕЗИРОВАНИЕ_АОРТЫ']

In [5]:
oper_data = data.loc[data['appoint'].isin(oper_to_mod)] #Выделяем только нужные операции
oper_data = oper_data.loc[data['document'].isin(intra_docs)] #Выделяем только нужные операции

In [6]:
oper_data.document.value_counts()

ПРОТОКОЛ_ОПЕРАЦИИ.                            1427
ПРОТОКОЛ_ОПЕРАЦИИ_НА_СЕРДЦЕ                   1190
ПРОТОКОЛ_ОПЕРАЦИИ                              828
ПРОТОКОЛ_ОПЕРАЦИИ_ЭНДОПРОТЕЗИРОВАНИЕ_АОРТЫ      78
Name: document, dtype: int64

In [7]:
#Количество пациентов
len(list(unique_everseen(oper_data.patient)))

455

In [8]:
oper_data.head()

,index,patient,episode,date,time,clinitian,depart,hosp,specialty,appoint,document,text,vyp
359,404,GACAAGc,GACAAGcAARAAP13:17AaAD,2013-12-17,13:17,Артюшин_Борис_Сергеевич,НИЛСХ,Центр_Алмазова,Старший_научный_сотрудник_(в_области_медицины),Резекция_аневризмы_и_протезирование_аорты,ПРОТОКОЛ_ОПЕРАЦИИ.,основная_ведущая Операция РЕЗЕКЦИЯ ИНФРАРЕНАЛЬ...,выполнено
360,405,GACAAGc,GACAAGcAARAAP13:17AaAG,2013-12-17,13:17,Артюшин_Борис_Сергеевич,НИЛСХ,Центр_Алмазова,Старший_научный_сотрудник_(в_области_медицины),Резекция_аневризмы_и_протезирование_аорты,ПРОТОКОЛ_ОПЕРАЦИИ.,Описание операции Выполнен доступ к абдоминал...,выполнено
361,406,GACAAGc,GACAAGcAARAAP13:17AaAJ,2013-12-17,13:17,Артюшин_Борис_Сергеевич,НИЛСХ,Центр_Алмазова,Старший_научный_сотрудник_(в_области_медицины),Резекция_аневризмы_и_протезирование_аорты,ПРОТОКОЛ_ОПЕРАЦИИ.,,выполнено
362,407,GACAAGc,GACAAGcAARAAP13:17AaAM,2013-12-17,13:17,Артюшин_Борис_Сергеевич,НИЛСХ,Центр_Алмазова,Старший_научный_сотрудник_(в_области_медицины),Резекция_аневризмы_и_протезирование_аорты,ПРОТОКОЛ_ОПЕРАЦИИ.,Оператор Артюшин_Борис_Сергеевич,выполнено
363,408,GACAAGc,GACAAGcAARAAP13:17AaAP,2013-12-17,13:17,Артюшин_Борис_Сергеевич,НИЛСХ,Центр_Алмазова,Старший_научный_сотрудник_(в_области_медицины),Резекция_аневризмы_и_протезирование_аорты,ПРОТОКОЛ_ОПЕРАЦИИ.,Ассистент Юсифов_Рашад_Имран_Оглы,выполнено


Аггрегируем тексты операций по пациентам и времени (т.к. у одного пациента могло быть несколько операций)

In [9]:
oper_text = pd.DataFrame(oper_data.groupby(['patient','time']).agg(lambda x: list(x))['text']).reset_index()
oper_text.text = oper_text.text.apply(lambda x: ' '.join(x))
oper_text

,patient,time,text
0,GACAAGc,13:17,основная_ведущая Операция РЕЗЕКЦИЯ ИНФРАРЕНАЛЬ...
1,GACAAbN,21:42,основная_ведущая / Операция Операция Рестерно...
2,GACACET,18:08,"Операция Стернотомия, протезирование корня и в..."
3,GACACuN,13:33,основная_ведущая / Операция TEVAR / Дата нача...
4,GACADLz,14:35,"основная_ведущая Операция Рестернотомия, надко..."
...,...,...,...
485,GACAdio,15:19,основная_ведущая / ДИАГНОЗ ВЫПИСКИ_ОСНОВНОЙ I...
486,GACAd}e,14:14,основная_ведущая / Основной Аневризма восходя...
487,GACAdОj,16:57,основная_ведущая / ДИАГНОЗ ПОСТУПЛЕНИЯ_ОСНОВН...
488,GACAdжХ,17:19,основная_ведущая / ДИАГНОЗ ПРЕДОПЕРАЦИОННЫЙ_О...


In [10]:
text = list(oper_text.text)
len(text) #Количество записей с операциями

490

In [11]:
from natasha import NamesExtractor, MorphVocab, Doc, NewsNERTagger, NewsEmbedding, Segmenter, NewsMorphTagger, NewsSyntaxParser
morph_vocab = MorphVocab()
names_extractor = NamesExtractor(morph_vocab)
emb = NewsEmbedding()
segmenter = Segmenter()
ner_tagger = NewsNERTagger(emb)
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

def get_spans(text, segmenter, morph_tagger, morph_vocab, syntax_parser,ner_tagger):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    return doc.spans 

def clean_text(text, segmenter, morph_tagger, morph_vocab, syntax_parser,ner_tagger):
    spans = get_spans(text, segmenter, morph_tagger, morph_vocab, syntax_parser,ner_tagger)
    names=[]
    for span in spans:
        names.append(span.text)
    for name in names:
        text = re.sub(name, '', text)
    return text

Предобработка текстов

In [12]:
print('Удаляем имена')
new_text = [re.sub('_', ' ', item) for item in text]
new_text = [clean_text(item, segmenter, morph_tagger, morph_vocab, syntax_parser,ner_tagger) for item in tq(text)]
print('Чистим от символов')
letters_only = [re.sub("[^а-яА-Я]", " ", item) for item in text]
print('Еще раз имена')
new_text = [clean_text(item, segmenter, morph_tagger, morph_vocab, syntax_parser,ner_tagger) for item in tq(letters_only)]
print('Регистр и стоп-слова')
words = [item.lower().split() for item in new_text]
stops = set(stopwords.words("russian"))  
new_words=[]
for item in words:
    meaningful_words = [w for w in item if not w in stops]  
    new_words.append(meaningful_words)
print('Нормализация')
norm_words = [normalize_corp(item) for item in tq(new_words)]
texts = [" ".join(item) for item in norm_words]

  0%|          | 1/490 [00:00<00:52,  9.37it/s]

Удаляем имена


  0%|          | 0/490 [00:00<?, ?it/s]

Чистим от символов
Еще раз имена


  1%|          | 3/490 [00:00<00:19, 24.46it/s]

Регистр и стоп-слова
Нормализация


100%|██████████| 490/490 [00:21<00:00, 23.26it/s]


In [137]:
#processed_words = " ".join(texts).split()

In [13]:
from sklearn.cluster import KMeans
import networkx as nx
from node2vec import Node2Vec
import matplotlib.pyplot as plt
def plot_cooccur_graph(cooccur_dict, frq_threshold=0, remove_threshold=None, cluster_num=None,
                       plot=True, return_graph=False):
    """
    cooccur_dict: частотный словарь попарного упоминания хештегов
    frq_threshold: порог для отсечения низкочастотных пар
    remove_threshold: степень вершины, меньше которой она удаляется
    cluster_num: число кластеров (если None, то выводится результат без кластеризации)
    plot: отрисовка графа (bool)
    return_graph: возвращает граф (bool)
    """
    G = nx.Graph()
    for k, v in cooccur_dict.items():
        if v > frq_threshold:
            G.add_edge(k[0], k[1], weight=(v * 50))
    
    if remove_threshold:
        remove = [node for node,degree in dict(G.degree()).items() if degree < remove_threshold]
        G.remove_nodes_from(remove)
        remove_pendants = [node for node,degree in dict(G.degree()).items() if degree == 0] # "висячие" вершины
        G.remove_nodes_from(remove_pendants)
    
    if cluster_num:
        G1 = nx.convert_node_labels_to_integers(G)
        node2vec = Node2Vec(G1, dimensions=3, walk_length=20, num_walks=10, workers=2)
        model = node2vec.fit(window=10, min_count=1)
        vectors = model.wv.vectors
        indices = np.array([int(i) for i in model.wv.index_to_key], ndmin=2).T
        X = np.concatenate((indices, vectors), axis=1)
        X = X[X[:,0].argsort()]
        Z = X[0:X.shape[0],1:X.shape[1]]
        kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(Z)
        labels = kmeans.labels_
        cluster_result = dict(zip(list(G.nodes), labels))
        
    if plot:
        fig, ax = plt.subplots(figsize=(50, 50))
        pos = nx.spring_layout(G, k=3)
        if cluster_num:
            nx.draw_networkx(G, pos,
                             font_size=16,
                             width=3,
                             edge_color='grey',
                             node_color=list(cluster_result.values()),
                             cmap=plt.cm.tab20,
                             with_labels = False,
                             ax=ax)
        else:
            nx.draw_networkx(G, pos,
                             font_size=16,
                             width=3,
                             edge_color='grey',
                             node_color='purple',
                             with_labels = False,
                             ax=ax)
        for key, value in pos.items():
            x, y = value[0]+.035, value[1]+.025
            ax.text(x, y,
                    s=key,
                    bbox=dict(facecolor='red', alpha=0.25),
                    horizontalalignment='center', fontsize=13)
        plt.show();
    
    if return_graph:
        return G

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def tfIdf_score_filter(docs, threshold=0):
    """Фильтрация слов (токенов) по tf-idf score."""
    filtered_docs = []
    tfIdfVectorizer=TfidfVectorizer(use_idf=False)
    tfIdf = tfIdfVectorizer.fit_transform(docs)
    for i,d in enumerate(docs):
        tfIdf_scores = np.asarray(tfIdf[i].todense())[0]
        filtered_words = np.array(tfIdfVectorizer.get_feature_names())[tfIdf_scores > threshold]
        filtered_docs.append(list(filtered_words))
    return filtered_docs

In [19]:
filtered_tags = tfIdf_score_filter(texts, threshold=0.1)
filtered_tags

[['анастомоз',
  'аневризм',
  'аорта',
  'артерия',
  'брыжеечный',
  'время',
  'выделить',
  'дата',
  'линейный',
  'мешок',
  'наложить',
  'наркоз',
  'начало',
  'нижний',
  'окончание',
  'операция',
  'пережать',
  'подвздошный',
  'почечный',
  'протезирование',
  'смотреть'],
 ['алмазов',
  'анастомоз',
  'аорта',
  'аортальный',
  'артерия',
  'аутовенозный',
  'выделить',
  'выполнить',
  'евгениевич',
  'клапан',
  'мм',
  'операция',
  'отдел',
  'перфузия',
  'правый',
  'проксимальный',
  'протез',
  'протезирование',
  'сосудистый',
  'стенка',
  'центр',
  'церебральный',
  'шунт'],
 ['ак',
  'ан',
  'аорта',
  'артерия',
  'корень',
  'коронарный',
  'мм',
  'мн',
  'операция',
  'полость',
  'протез',
  'сосудистый',
  'ст',
  'створка',
  'устьевый'],
 ['анестезия',
  'артерия',
  'время',
  'графт',
  'дата',
  'доступ',
  'катетер',
  'катетеризация',
  'кровоток',
  'левый',
  'область',
  'операция',
  'подключичный',
  'правый',
  'проводник',
  'пункция',
  

In [20]:
from itertools import combinations

word_cooccurrence = dict()
all_cooccurrences = []
for d in tq(filtered_tags):
    d_cooccur = list(combinations(d,2))
    for cooccur in d_cooccur:
        if set(cooccur) not in all_cooccurrences:
            all_cooccurrences.append(set(cooccur))
            word_cooccurrence[cooccur] = 0
        try:
            word_cooccurrence[cooccur] += 1
        except:
            reverse_cooccur = (cooccur[-1],cooccur[0])
            word_cooccurrence[reverse_cooccur] += 1

100%|██████████| 490/490 [00:13<00:00, 36.36it/s]


In [ ]:
plot_cooccur_graph(word_cooccurrence, frq_threshold=3, cluster_num=4)